In [32]:
def crop_center(img):
  width, height = img.shape[1], img.shape[0]
  crop_width = min(img.shape[0], img.shape[1])
  crop_height = min(img.shape[0], img.shape[1])
  mid_x, mid_y = int(width/2), int(height/2)
  cw2, ch2 = int(crop_width/2), int(crop_height/2) 
  crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
  return crop_img


In [56]:
dataset, dimensions, svm_name, kmeans_name = (
  "groceries", 
  (224, 224), 
  "new_models/sift_kmeans_svm_groceries_224x224_f32.dat",
  "new_models/sift_kmeans_svm_groceries_centers_224x224_f32.npy"
)

# dataset, dimensions, svm_name, kmeans_name = (
#   "cifar-10", 
#   (32, 32),
#   "new_models/sift_kmeans_svm_cifar_32x32_f32.dat",
#   "new_models/sift_kmeans_svm_cifar_centers_32x32_f32.npy"
# )

# dataset, dimensions, svm_name, kmeans_name = (
#   "Petimages", 
#   (150, 150),
#   "new_models/sift_kmeans_svm_petimages_150x150_f32.dat",
#   "new_models/sift_kmeans_svm_petimages_centers_150x150_f32.npy"
# )

# Load train and test data

In [57]:
import numpy as np
import cv2
from pathlib import Path
from sklearn.utils import Bunch

def load_image_files(images_dir_path, test_size=0.2, target_size=(224, 224),):
    images_dir = Path(images_dir_path)
    directories = [directory for directory in images_dir.iterdir()]
    classes = [class_directory.name.lower() for class_directory in directories]

    train_data = []
    test_data = []

    for class_index, directory in enumerate(directories):
        if not directory.is_dir():
            print("is not directory", directory.as_posix())
            continue

        class_images_cnt = 0
        for file in directory.iterdir():
            class_images_cnt += 1

        sift = cv2.SIFT_create()
        test_size_cnt = int(class_images_cnt * test_size)
        for i, file in enumerate(directory.iterdir()):
            if file.is_dir():
                print("directory", directory.as_posix())
                continue

            image = cv2.imread(file.as_posix(), cv2.IMREAD_COLOR)
            if image is None:
                print("bad file: ", file.as_posix())
                continue

            width, height = image.shape[1], image.shape[0]
            if width == 0 or height == 0:
                print("bad file: ", file.as_posix())
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = crop_center(image)
            image = cv2.resize(image, target_size)
            kp, des = sift.detectAndCompute(image, None)

            batch = Bunch(
                # image=image,
                class_index=class_index,
                class_name=classes[class_index],
                des = des,
                kp = kp,
            )

            if i < test_size_cnt:
                test_data.append(batch)
                continue

            train_data.append(batch)

    return train_data, test_data


train_dataset, test_dataset = load_image_files(dataset, target_size=dimensions)

is not directory groceries/.DS_Store


# Render sample train image

In [4]:
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# random_index = np.random.randint(len(train_dataset))

In [5]:
# plt.imshow(train_dataset[random_index].image.astype(int))
# plt.show()

# Extract train and test data from batches

In [58]:
import numpy as np

# descriptors_stack = np.vstack([batch.des for batch in train_dataset])
descriptors = []
for batch in train_dataset:
    if batch.des is not None:
        descriptors.append(batch.des)
descriptors_stack = np.vstack(descriptors)

# Apply k-means clustering

In [59]:
num_clusters = 200
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
flags = cv2.KMEANS_RANDOM_CENTERS
_, cluster_ids, cluster_centers = cv2.kmeans(descriptors_stack, num_clusters, None, criteria, 10, flags)

# Calculate histograms for each image in train dataset

In [60]:
def calculate_histogram(cluster_centers, image_descriptor):
  histogram = np.zeros(len(cluster_centers))
  if image_descriptor is None:
    return histogram
  
  for descriptor in image_descriptor:
      distances = np.linalg.norm(descriptor - cluster_centers, axis=1)
      nearest_cluster_index = np.argmin(distances)
      histogram[nearest_cluster_index] += 1

  return histogram


def calc_histograms_progress(dataset, cluster_centers):
  histograms = []
  last_percent = -1
  for i in range(len(dataset)):
    histograms.append(calculate_histogram(cluster_centers, dataset[i].des))

    percent_candidate = int((i / len(dataset)) * 100)
    if percent_candidate > last_percent:
      print(f"{percent_candidate}%")
      last_percent = percent_candidate

  return np.asarray(histograms)


x_train = calc_histograms_progress(train_dataset, cluster_centers)
y_train = np.asarray([batch.class_index for batch in train_dataset])

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


# Learn SVM on features extracted

In [61]:
import cv2

svm = cv2.ml.SVM_create()
svm.setType(cv2.ml.SVM_C_SVC)
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 500, 1e-6))
svm.train(x_train.astype(np.float32), cv2.ml.ROW_SAMPLE, y_train)

True

In [62]:
from sklearn.metrics import classification_report

x_test = calc_histograms_progress(test_dataset, cluster_centers)
y_test = np.asarray([batch.class_index for batch in test_dataset])

_, y_pred = svm.predict(x_test.astype(np.float32))
print(classification_report(y_test, y_pred))

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
              precision    recall  f1-score   support

           0       0.34      0.43      0.38        23
           1       0.26      0.43      0.33        21
           3       0.19      0.24      0.21        33
           4       0.14      0.13      0.14        52
           5       0.16      0.17      0.16        30
           6       0.25      0.28      0.26        36
           7       0.24      0.24      0.24        34
           8       0.27      0.25      0.26        32
           9       0.13      0.16      0.14        31
          10       0.24      0.23      0.24        61
          11   

In [63]:
svm.save(svm_name)
np.save(kmeans_name, cluster_centers)

In [ ]:
svm = cv2.ml.SVM_load(svm_name)
cluster_centers = np.load(kmeans_name)

x_test = calc_histograms_progress(test_dataset, cluster_centers)
y_test = np.asarray([batch.class_index for batch in test_dataset])
_, y_pred = svm.predict(x_test.astype(np.float32))
print(classification_report(y_test, y_pred))

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
              precision    recall  f1-score   support

           0       0.08      0.09      0.09        33
           1       0.24      0.27      0.25        52
           2       0.22      0.27      0.24        37
           3       0.15      0.19      0.17        31
           4       0.30      0.27      0.28        30
           5       0.23      0.37      0.28        19
           6       0.33      0.34      0.34        32
           7       0.16      0.15      0.16        59
           8       0.20      0.27      0.23        22
           9       0.15      0.12      0.13        60
          10   